# 1导入库与数据

In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
file_path = r'C:\Users\a\.spyder-py3\tf-notebooks\residential_electricity'
#Windows系统中，路径中的反斜杠\是转义字符，所以你需要使用双反斜杠\\或者前置r来创建一个原始字符串。
os.chdir(file_path)
df = pd.read_csv("data.csv",parse_dates = [1],index_col= [0,1])
#index_col设置双重索引
df.head()

KWH
CONS_NO DATA_DATE       
1       2015-01-01  6.68
        2015-01-02  2.50
        2015-01-03  5.20
        2015-01-04  4.17
        2015-01-05  4.89

In [3]:
replace_dict = {"NA":np.nan,np.nan:np.nan}
df['KWH'].fillna(replace_dict,inplace = True)

In [4]:
df.isnull().sum()

KWH    1018
dtype: int64

# 2数据预处理

In [5]:
df = df.unstack().KWH
#对内层索引转换为列，通常用于时间序列数据将KWH这列作为结果

In [6]:
type(df.iloc[1,1])

numpy.float64

In [7]:
df.head()

DATA_DATE,2015-01-01,2015-01-02,2015-01-03,2015-01-04,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-10,...,2017-01-29,2017-01-30,2017-01-31,2017-02-01,2017-02-02,2017-02-03,2017-02-04,2017-02-05,2017-02-06,2017-02-07
CONS_NO,,,,,,,,,,,,,,,,,,,,,
1,6.68,2.50,5.20,4.17,4.89,5.26,4.11,3.70,3.74,5.83,...,2.37,3.12,2.02,1.74,2.53,3.18,3.29,2.77,3.81,3.62
2,1.22,0.65,1.14,1.04,1.33,1.02,0.08,0.09,0.09,0.09,...,0.97,0.95,0.93,0.92,0.92,1.99,0.96,1.36,1.33,1.40
3,7.35,6.65,7.76,4.02,4.68,7.06,6.51,7.00,6.53,4.94,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.62,1.92,0.65,1.30,0.71,1.36,0.89,1.24,0.75,1.67,...,1.14,1.06,0.96,1.12,0.86,0.76,0.75,1.49,0.86,1.08
5,2.58,2.60,2.36,1.83,2.05,1.97,1.98,1.44,2.02,2.33,...,2.11,2.58,2.22,1.96,1.71,2.23,2.69,2.27,2.52,2.10


## 异常值

In [8]:
for col in range(df.shape[1]):
    col_mean = df.iloc[:,col].mean()
    col_std =df.iloc[:,col].std()
    mask = (df.iloc[:,col]-col_mean).abs()>3*col_std
    df.iloc[:,col][mask] = np.nan
    #根据条件矩阵进行赋值时，只有布尔矩阵中对应位置为 True 的元素会被替换。

In [9]:
df.head()

DATA_DATE,2015-01-01,2015-01-02,2015-01-03,2015-01-04,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-10,...,2017-01-29,2017-01-30,2017-01-31,2017-02-01,2017-02-02,2017-02-03,2017-02-04,2017-02-05,2017-02-06,2017-02-07
CONS_NO,,,,,,,,,,,,,,,,,,,,,
1,6.68,2.50,5.20,4.17,4.89,5.26,4.11,3.70,3.74,5.83,...,2.37,3.12,2.02,1.74,2.53,3.18,3.29,2.77,3.81,3.62
2,1.22,0.65,1.14,1.04,1.33,1.02,0.08,0.09,0.09,0.09,...,0.97,0.95,0.93,0.92,0.92,1.99,0.96,1.36,1.33,1.40
3,7.35,6.65,7.76,4.02,4.68,7.06,6.51,7.00,6.53,4.94,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.62,1.92,0.65,1.30,0.71,1.36,0.89,1.24,0.75,1.67,...,1.14,1.06,0.96,1.12,0.86,0.76,0.75,1.49,0.86,1.08
5,2.58,2.60,2.36,1.83,2.05,1.97,1.98,1.44,2.02,2.33,...,2.11,2.58,2.22,1.96,1.71,2.23,2.69,2.27,2.52,2.10


In [10]:
df.max(axis =1)

CONS_NO
1      25.16
2      15.46
3      27.00
4       4.70
5      16.50
       ...  
196     6.28
197    50.40
198    24.89
199    11.83
200     0.00
Length: 200, dtype: float64

In [11]:
df.agg(['max','min','median','sum','var','skew','kurt'],axis=1)

,max,min,median,sum,var,skew,kurt
CONS_NO,,,,,,,
1,25.16,0.0,3.120,3065.93,11.762377,3.040316,11.211566
2,15.46,0.0,1.790,1695.10,3.926000,3.420053,14.341789
3,27.00,0.0,3.600,2803.25,14.945491,1.582819,5.123261
4,4.70,0.0,1.000,860.84,0.301526,1.763758,4.600405
5,16.50,0.0,2.270,2374.45,6.194531,2.502617,6.188802
...,...,...,...,...,...,...,...
196,6.28,0.0,1.170,894.53,0.974862,2.133523,7.439265
197,50.40,0.0,0.765,2194.90,51.468509,3.810612,15.842375
198,24.89,0.0,3.665,2770.84,9.502098,3.238847,14.851859


## 差分处理

In [12]:
df.diff(axis = 1).agg(['max','min','median','sum','var','skew','kurt'])

DATA_DATE,2015-01-01,2015-01-02,2015-01-03,2015-01-04,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-10,...,2017-01-29,2017-01-30,2017-01-31,2017-02-01,2017-02-02,2017-02-03,2017-02-04,2017-02-05,2017-02-06,2017-02-07
max,NaN,5.360000,5.710000,5.320000,3.850000,8.880000,5.200000,4.870000,3.440000,4.910000,...,10.510000,4.360000,5.060000,11.640000,6.590000,6.350000,7.580000,5.100000,4.920000,11.350000
min,NaN,-4.200000,-5.200000,-5.460000,-4.910000,-3.380000,-6.180000,-4.240000,-6.420000,-8.480000,...,-6.460000,-5.020000,-5.430000,-3.050000,-9.770000,-9.450000,-5.900000,-5.380000,-9.090000,-6.900000
median,NaN,0.000000,0.000000,0.000000,0.000000,0.005000,-0.030000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sum,0.0,15.080000,25.160000,-26.600000,-17.810000,54.800000,-47.320000,-14.560000,-2.550000,11.540000,...,-33.000000,-19.170000,-8.730000,68.630000,-4.800000,28.860000,3.660000,-10.950000,-7.260000,19.110000
var,NaN,1.529140,1.453966,1.842255,1.289004,1.510381,1.355740,0.969286,1.374143,1.537225,...,1.944687,1.133146,1.205654,2.863745,2.644233,2.447967,1.896841,1.449804,1.864745,2.147470
skew,NaN,0.664766,0.313423,-0.073182,-0.137913,2.217920,-0.272209,-0.125547,-1.302175,-1.697730,...,1.474367,-0.546191,-0.335455,3.442935,-2.190073,-0.754902,0.657414,-0.006649,-1.652249,2.127068
kurt,NaN,4.612046,4.160558,3.234514,3.750176,13.244584,6.564531,5.967313,8.744256,12.871007,...,20.783532,6.768244,8.153275,17.756790,13.953656,9.051757,6.685429,4.797595,11.431051,21.127475


## 分位数计算

In [13]:
df_quantile = df.quantile([.05],axis = 1)

In [14]:
df_quantile

CONS_NO,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
0.05,1.63,0.6905,0.0,0.58,1.5005,2.12,0.7625,1.107,0.0,0.0,...,0.0605,1.614,0.04,0.0,0.0205,0.0,0.0,1.13,0.15,0.0


# 3分析
## 按周求和

In [15]:
df.columns.isocalendar().week

DATA_DATE
2015-01-01    1
2015-01-02    1
2015-01-03    1
2015-01-04    1
2015-01-05    2
             ..
2017-02-03    5
2017-02-04    5
2017-02-05    5
2017-02-06    6
2017-02-07    6
Name: week, Length: 746, dtype: UInt32

In [16]:
print(df.dtypes)

DATA_DATE
2015-01-01    float64
2015-01-02    float64
2015-01-03    float64
2015-01-04    float64
2015-01-05    float64
               ...   
2017-02-03    float64
2017-02-04    float64
2017-02-05    float64
2017-02-06    float64
2017-02-07    float64
Length: 746, dtype: object


In [17]:
df_week_diff =df.groupby(df.columns.isocalendar().week,axis=1).sum().diff(axis =1)

In [18]:
df_week_diff

week,1,2,3,4,5,6,7,8,9,10,...,44,45,46,47,48,49,50,51,52,53
CONS_NO,,,,,,,,,,,,,,,,,,,,,
1,NaN,13.32,-9.07,6.56,3.82,-23.02,-14.09,-7.93,13.87,1.44,...,-5.12,10.93,2.46,2.06,2.11,-3.89,-0.52,3.99,-4.67,-14.57
2,NaN,-0.36,2.79,1.40,2.16,-20.77,3.14,-5.35,-2.57,7.79,...,-3.41,6.38,1.24,-1.40,1.49,0.01,-1.59,-1.79,0.22,-13.07
3,NaN,13.99,9.44,-4.62,6.38,-29.93,29.13,-16.67,3.21,9.24,...,-2.74,40.38,-5.05,-18.38,0.91,2.75,3.76,-13.56,1.55,2.45
4,NaN,2.53,1.29,-0.23,-1.79,-4.59,-3.75,7.18,-1.76,-2.52,...,-2.76,4.19,1.34,-1.85,3.70,-3.27,-1.23,2.85,-4.58,-3.23
5,NaN,3.87,0.23,1.94,1.86,-20.07,0.63,4.84,-7.60,1.69,...,-2.80,11.52,0.37,-1.72,-2.11,0.69,1.28,2.61,-4.77,-13.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,NaN,0.58,3.32,-20.53,-1.69,-8.31,-3.20,5.63,5.81,0.56,...,-3.61,12.75,-5.16,-1.19,6.42,-6.45,-0.51,5.29,0.18,-11.12
197,NaN,25.92,-9.47,3.33,-28.55,-10.48,3.13,-1.02,-2.29,0.45,...,-8.11,17.01,-14.12,4.29,34.12,-0.94,-14.37,25.47,-4.43,-51.54
198,NaN,10.00,-13.66,-2.75,-20.19,-20.37,-10.69,-5.39,17.29,6.99,...,-0.14,2.77,-12.56,28.92,-4.63,5.16,-0.47,3.94,-2.63,-25.87


In [19]:
df_week_diff.agg(['max','min','mean','median','skew','kurt'])

week,1,2,3,4,5,6,7,8,9,10,...,44,45,46,47,48,49,50,51,52,53
max,NaN,60.760000,50.400000,41.300000,56.880000,25.010000,29.130000,38.380000,23.140000,42.210000,...,46.980000,82.380000,65.500000,69.860000,34.120000,16.790000,43.620000,45.640000,30.430000,5.240000
min,NaN,-51.960000,-38.210000,-52.830000,-37.750000,-99.370000,-43.860000,-34.810000,-30.110000,-34.760000,...,-37.280000,-30.080000,-52.690000,-84.190000,-64.180000,-56.510000,-49.350000,-24.620000,-45.270000,-76.250000
mean,NaN,8.057200,0.496950,-3.114750,0.059400,-22.179400,-0.401800,-1.253350,0.452600,1.556450,...,-3.903250,16.441600,-0.764750,-3.197100,-0.991550,-0.793350,-0.239950,1.209250,-2.892000,-18.506950
median,NaN,7.235000,0.090000,-2.160000,-0.220000,-17.615000,0.000000,-0.290000,-0.095000,0.465000,...,-3.340000,12.800000,-0.240000,-1.155000,0.000000,0.000000,0.000000,0.265000,-2.420000,-15.570000
skew,NaN,0.405748,0.810921,-0.514372,0.547050,-1.224725,-0.831454,0.218368,-0.016266,0.890321,...,1.169533,1.117186,0.825877,-0.618119,-2.318047,-3.156778,-0.580143,1.186734,-0.787911,-1.228664
kurt,NaN,6.751138,4.503521,2.971495,2.501935,1.559414,2.755416,2.027523,1.612258,5.676722,...,9.499920,2.161546,6.206592,7.278628,19.386874,23.443914,11.384289,5.396189,7.773246,1.927245


## 求出每户居民用电量大于0.9倍最大值的天数

In [20]:
result_list = []
for row in range(df.shape[0]):
    df_rmax =df.iloc[row,:] .max()
    mask_1 =df.iloc[row,:]>0.9*df_rmax
    result = df.iloc[row,:][mask_1].count().sum()
    result_list.append(result)

In [21]:
df_09 = pd.DataFrame(result_list)
df_09.head()

,0
0,5
1,5
2,2
3,1
4,2


### 更简单的方法

In [22]:
df_per10 = df.max(axis = 1)*0.9

In [23]:
df[df.sub(df_per10,axis =0)>0].count(axis = 1)
#sub减法

CONS_NO
1      5
2      5
3      2
4      1
5      2
      ..
196    6
197    2
198    2
199    1
200    0
Length: 200, dtype: int64

## 求每个用电数据最大值和最小值出现的月份

In [24]:
#求出每月的最大值
df_maxmonth = df.groupby(df.columns.to_period('M'),axis = 1).max()
df_maxmonth

DATA_DATE,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,...,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02
CONS_NO,,,,,,,,,,,,,,,,,,,,,
1,8.37,5.43,4.89,4.88,4.01,4.66,25.11,23.72,5.30,5.46,...,4.31,17.03,25.16,19.82,4.44,3.82,7.79,4.86,4.96,3.81
2,3.80,2.38,4.84,2.52,2.32,4.32,15.46,14.04,3.23,3.20,...,2.95,14.79,10.39,14.06,3.54,2.96,3.36,2.94,5.55,1.99
3,8.50,7.07,12.34,14.78,10.42,11.11,27.00,26.01,5.29,6.42,...,0.01,0.00,0.00,1.21,1.23,1.23,0.06,0.00,0.03,0.00
4,2.03,3.57,2.05,1.47,2.35,2.38,1.95,2.60,2.26,1.68,...,3.02,2.94,2.58,3.73,2.68,2.94,2.25,1.68,3.02,1.49
5,2.96,4.46,3.36,2.82,2.80,4.47,13.06,16.50,5.96,2.73,...,2.82,11.20,13.14,14.99,5.81,2.98,3.27,2.89,3.48,2.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,5.43,1.51,6.02,5.86,6.28,2.68,5.18,2.53,4.84,5.09,...,4.16,4.75,1.92,1.54,1.76,3.31,1.99,1.71,4.03,0.00
197,0.00,0.00,0.00,1.69,6.57,0.04,1.13,7.67,1.25,2.14,...,11.27,31.49,36.02,50.40,9.70,4.24,11.24,15.60,17.31,2.50
198,7.27,4.61,5.60,1.33,5.48,6.30,13.78,16.45,5.60,5.85,...,4.51,13.07,12.37,24.89,7.21,1.30,4.98,5.27,5.46,1.28


In [40]:
df_maxmonth.iloc[1,:][(df_maxmonth.iloc[1,:] == df_maxmonth.iloc[1,:].max())]

DATA_DATE
2015-07    15.46
Freq: M, Name: 2, dtype: float64

In [46]:
date = []
value = []
for row in range(df_maxmonth.shape[0]):
    ind1 = df_maxmonth.iloc[row,:][(df_maxmonth.iloc[row,:] == df_maxmonth.iloc[row,:].max())].index
    #提取出最大值的月份
    if len(ind1) == 1:#只包含一个最大值月份时
        max_month_column = df.columns[df.columns.to_period('M') == ind1[0]]
        count1 = pd.DataFrame([df.iloc[row, :][max_month_column] == df_maxmonth.iloc[row,:].max()]).sum(axis = 1)
        #包含最大值的个数
        result_value = count1.values[0]
        #得到取值
        #提取月份
        date.append(ind1[0])
        value.append(result_value)
    else:
        month_dict = {}
        for i in range(len(ind1)):
            max_month_column = df.columns[df.columns.to_period('M') == ind1[i]]
            count1 = pd.DataFrame([df.iloc[row, :][max_month_column] == df_maxmonth.iloc[row,:].max()]).sum(axis = 1)
            result_value = count1.values[0]
            month_dict[ind1[i]] = result_value
            #生成字典
            max_key = max(month_dict, key=month_dict.get)
            #max(my_dict, key=my_dict.get) 返回字典 my_dict 中具有最大值的键，其中 key=my_dict.get 表示比较的标准是字典中键对应的值。
            #获得DataFrme最大值对应的索引是idindex()方法
            max_month_column = df.columns[df.columns.to_period('M') == max_key]
            date.append(max_month_column)
            value.append(result_value)
data = {'DATA_DATE':date,'count':value}
        #通过字典生成datdaframe
df_1 = pd.DataFrame(data)
df_1.head(200)
    

,DATA_DATE,count
0,2016-07,1
1,2015-07,1
2,2015-07,1
3,2015-11,1
4,2015-08,1
...,...,...
195,2017-01,1
196,2016-07,1
197,2017-01,1
198,2015-07,1


## 求取每个用户七八月电量和与三四月电量和的比值，最大值的比值，最小值的比值，均值（日均电量）的比值

In [60]:
elec78 = df.groupby(df.columns.month,axis = 1)
elec78.head()

DATA_DATE,2015-01-01,2015-01-02,2015-01-03,2015-01-04,2015-01-05,2015-02-01,2015-02-02,2015-02-03,2015-02-04,2015-02-05,...,2015-11-01,2015-11-02,2015-11-03,2015-11-04,2015-11-05,2015-12-01,2015-12-02,2015-12-03,2015-12-04,2015-12-05
CONS_NO,,,,,,,,,,,,,,,,,,,,,
1,6.68,2.50,5.20,4.17,4.89,4.14,4.27,4.12,4.29,4.34,...,2.68,3.12,2.19,4.18,2.92,1.99,2.91,3.37,3.65,3.19
2,1.22,0.65,1.14,1.04,1.33,2.19,1.79,2.01,2.38,1.61,...,2.19,1.52,1.97,2.20,1.27,1.75,1.50,1.96,2.04,1.34
3,7.35,6.65,7.76,4.02,4.68,6.27,5.00,7.07,4.93,5.47,...,5.04,9.56,8.37,NaN,11.49,6.71,6.35,8.06,9.08,5.39
4,0.62,1.92,0.65,1.30,0.71,1.06,1.02,1.88,0.94,0.99,...,0.88,0.64,1.05,0.94,0.93,0.78,1.00,1.26,1.24,0.55
5,2.58,2.60,2.36,1.83,2.05,1.98,2.58,1.72,2.00,1.61,...,1.90,2.41,2.36,1.95,3.68,1.36,2.37,2.19,2.27,1.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,1.87,1.29,1.21,1.52,1.52,1.47,1.35,1.51,0.62,0.27,...,1.24,6.19,1.46,1.17,1.17,1.51,1.93,1.71,1.13,1.17
197,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.50,3.61,0.01,0.06,0.00
198,4.59,4.86,4.49,7.27,6.55,4.27,4.06,4.23,3.76,2.96,...,4.78,3.71,4.15,4.53,4.64,3.88,3.98,3.33,4.41,3.25


In [64]:
elec78 = df.groupby(df.columns.month,axis = 1).sum().iloc[:,6:8]
#使用 df.groupby(df.columns.month, axis=1) 后，你会得到一个按照月份分组的
elec34 = df.groupby(df.columns.month,axis = 1).sum().iloc[:,2:4]
df_elec_sum_scale = elec78.sum(axis = 1)/elec34.sum(axis=1)
df_elec_max_scale = elec78.max(axis = 1)/elec34.max(axis=1)
df_elec_min_scale = elec78.min(axis = 1)/elec34.max(axis=1)
df_elec_mean_scale = elec78.min(axis = 1)/elec34.max(axis=1)

In [71]:
type(df_elec_max_scale)
df_elec_max_scale

CONS_NO
1       3.085316
2       2.437826
3       0.820873
4       1.253876
5       3.530967
         ...    
196     0.656344
197    14.391973
198     2.945957
199     1.038961
200          NaN
Length: 200, dtype: float64

# 4将所有统计特征合并

In [74]:
features = pd.DataFrame([df_elec_max_scale,df_elec_mean_scale,df_elec_min_scale],index = ["df_elec_max_scale","df_elec_mean_scale","df_elec_min_scale"])
#对Series对象进行合并

In [75]:
features

CONS_NO,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
df_elec_max_scale,3.085316,2.437826,0.820873,1.253876,3.530967,2.346275,1.944936,2.334299,0.822345,0.192750,...,1.623356,1.427580,3.411714,2.947183,0.454349,0.656344,14.391973,2.945957,1.038961,NaN
df_elec_mean_scale,2.674308,2.040300,0.530352,0.891731,3.469183,1.485006,1.863156,1.692276,0.696434,0.185053,...,1.606084,0.979757,2.216115,2.843235,0.368557,0.224961,6.527760,1.300062,1.031039,NaN
df_elec_min_scale,2.674308,2.040300,0.530352,0.891731,3.469183,1.485006,1.863156,1.692276,0.696434,0.185053,...,1.606084,0.979757,2.216115,2.843235,0.368557,0.224961,6.527760,1.300062,1.031039,NaN
